In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The best answer is  Neil Armstrong


In [4]:


loader = WebBaseLoader("https://careers.nike.com/workday-integrations-software-engineer-iii/job/R-55167")
page_data = loader.load().pop().page_content
print(page_data)























Workday Integrations Software Engineer III










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to 

When extracting information from a webpage, all text data is retrieved. However, we only need specific, relevant data rather than the entire content. 

Therefore, we require a prompt designed to extract the exact specific information needed.

In [5]:


prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Workday Integrations Software Engineer III",
  "experience": [
    "5+ years' experience integrating systems with third-party service vendors",
    "5+ Workday Integrations with Workday Studio, EIB, Core Connectors and payroll connectors (PECI) and their related tools such as XML, XSLT",
    "At least one full implementation of Workday, with demonstrated Workday technical and integrations experience"
  ],
  "skills": [
    "Workday Studio",
    "EIB",
    "Core Connectors",
    "payroll connectors (PECI)",
    "XML",
    "XSLT",
    "Web Services (WSDL, SOAP)",
    "EDI",
    "Flat file integrations",
    "Object Oriented concepts",
    "Agile Framework"
  ],
  "description": "As a Workday Senior Software Integration Engineer III, you will work with the team of engineers working on integrations across products. You will be responsible to develop, test, and deploy integrations required to meet business requirements. You will be driving the technical deliverables to 

In [6]:
type(res.content)

str

It's an Text data. Now Converting text data to JSON object. <br>
Text ---> JSON

In [7]:


json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


{'role': 'Workday Integrations Software Engineer III',
 'experience': ["5+ years' experience integrating systems with third-party service vendors",
  '5+ Workday Integrations with Workday Studio, EIB, Core Connectors and payroll connectors (PECI) and their related tools such as XML, XSLT',
  'At least one full implementation of Workday, with demonstrated Workday technical and integrations experience'],
 'skills': ['Workday Studio',
  'EIB',
  'Core Connectors',
  'payroll connectors (PECI)',
  'XML',
  'XSLT',
  'Web Services (WSDL, SOAP)',
  'EDI',
  'Flat file integrations',
  'Object Oriented concepts',
  'Agile Framework'],
 'description': 'As a Workday Senior Software Integration Engineer III, you will work with the team of engineers working on integrations across products. You will be responsible to develop, test, and deploy integrations required to meet business requirements. You will be driving the technical deliverables to be delivered on time. Create detailed system integrati

In [8]:
type(json_res)

dict

In [9]:


df = pd.read_csv("Skill__related_Portfolio.csv")
df

,Requirement,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


Store the porfolios into vectordatabase

In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Requirement"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

Showing link related to the Skills

In [11]:
links = collection.query(query_texts=["Experiance in Javascripts", "Experiance in Java"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [12]:
job=json_res
job['skills']

['Workday Studio',
 'EIB',
 'Core Connectors',
 'payroll connectors (PECI)',
 'XML',
 'XSLT',
 'Web Services (WSDL, SOAP)',
 'EDI',
 'Flat file integrations',
 'Object Oriented concepts',
 'Agile Framework']

In [13]:
links= collection.query(query_texts=job['skills'],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'li

Now creating another prompt to generate email

In [14]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Faisal, CEO at HossainAI. HossainAI is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of HossainCo
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase HossainAI's portfolio: {link_list}
        Remember you are Faisal, CEO at HossainAI.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Workday Integrations Solution for Your Business Needs

Dear Hiring Manager,

I came across the job posting for a Workday Integrations Software Engineer III and was impressed by the requirements. As the CEO of HossainAI, I'd like to introduce our company and highlight our capabilities in delivering seamless integration solutions that cater to your business needs.

At HossainAI, we specialize in empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in integrating systems with third-party service vendors, particularly with Workday Studio, EIB, Core Connectors, and payroll connectors (PECI).

We have successfully delivered numerous integration projects, showcasing our expertise in:

* Workday Integrations with Workday Studio, EIB, Core Connectors, and payroll connectors (PECI) and their related tools such as XML, XSLT
* Web Services (WSDL, 